# Dataset Creation

Dans ce notebook, nous allons entrainer le premier modèle nécessaire à la création de notre outil de réduction de la difficulté.

In [1]:

# ---------------------------- PREPARING NOTEBOOK ---------------------------- #
# Autoreload
%load_ext autoreload
%autoreload 2

# Random seed
import numpy as np
np.random.seed(42)

# External modules
import os
from IPython.display import display

# Set global log level
import logging
logging.basicConfig(level=logging.INFO)
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# Define PWD as the current git repository
import git
repo = git.Repo('.', search_parent_directories=True)
pwd = repo.working_dir

# import

In [2]:
MODEL = "bofenghuang/vigostral-7b-chat"
# import

## $N$ simpler sentences

Dans un premier temps, nous allons demander à MISTRAL de nous générer $N$ variations simplifiée d'une phrase donnée. Pour ce faire, nous allons d'abord créer un jeu de donnée très simple, composé de phrases et de leurs simplifications qui seront utilisées pour fine-tuner le modèle. Pour nous aider dans la création de ce jeu de donnée, nous allons utiliser GPT-4, sans aucun fine-tuning, pour générer des simplifications de phrases.

### GPT-4 Fine-tuning Dataset

Pour créer notre premier jeu de donnée que nous appelerons `Fine-tuning dataset` par la suite, nous allons utiliser la prompt suivante avec le modèle GPT-4 :

```mkd
Dans le contexte d'un travail de recherche, je souhaite obtenir 5 exemples de simplifications d'une phrase donnée en français. Invente 6 phrases de niveau de difficulté CECRL respectifs A2, B1, B2, C1 et C2. Chaque phrase doit parler d'un sujet différent. Tu dois ensuite simplifier chacune de ces phrases en diminuant son niveau CECRL de 1 cran. Voici un exemple de simplification depuis le niveau C2 vers le niveau C1.

---
Phrase à simplifier de niveau CECRL C2 : 'Alors qu'au cours du Paléolithique, la diffusion avait fractionné l'écoumène en petits groupes, la tendance depuis la fin de la dernière glaciation est à l'épaississement des relations dans l'Ancien Monde, par augmentation des effectifs et par tissage d'interconnexions.'
Phrase simplifiée de niveau CECRL C1 : 'Pendant le Paléolithique, la diffusion avait séparé la population en petits groupes. Depuis la fin de la dernière glaciation, les relations se sont renforcées dans l'Ancien Monde, avec plus de personnes et un réseau de connexions plus étroit.'
---
!!! Tu ne dois pas utiliser cet exemple pour ton travail. Tu dois inventer tes propres phrases. !!!

Ton message doit être structuré de la manière suivante :
---
{Phrase de niveau CECRL A2} -> {Phrase simplifiée de niveau CECRL A1}\n
{Phrase de niveau CECRL B1} -> {Phrase simplifiée de niveau CECRL A2}\n
{Phrase de niveau CECRL B2} -> {Phrase simplifiée de niveau CECRL B1}\n
{Phrase de niveau CECRL C1} -> {Phrase simplifiée de niveau CECRL B2}\n
{Phrase de niveau CECRL C2} -> {Phrase simplifiée de niveau CECRL C1}\n
---
!!! Tu ne dois écrire que les phrases inventées et les phrases simplifiées. Tu ne dois ajouter aucun commentaire. !!!
```

---
Un jeu de donnée généré par GPT-4 est disponible dans le dossier `data/fine-tuning-dataset.csv`. Il est composé de 100 exemples de phrases et de leurs simplifications. Nous allons utiliser ce jeu de donnée pour fine-tuner notre modèle.

### Download the dataset 

In [3]:
# ----------------------- DOWNLOAD FINE-TUNING DATASET ----------------------- #
import pandas as pd
from huggingface_hub import snapshot_download


def ft_download_data(data_name: str, pwd: str = None):
    """
    Download the data from the HuggingFace Hub.

    Args:
        data_name (str): The name of the data to download.
        pwd (str): The current working directory. Defaults to None.
    """
    # Find PWD
    if pwd is None:
        repo = git.Repo(".", search_parent_directories=True)
        pwd = repo.working_dir

    # Determine the path
    if data_name == "sentence_simplification":
        path = os.path.join(pwd, "data", "raw")
        if not os.path.exists(path):
            os.makedirs(path)
    elif data_name == "Data":
        path = os.path.join(pwd, "data", "raw")
        if not os.path.exists(path):
            os.makedirs(path)
    else:
        raise ValueError(f"The data {data_name} is not available.")

    # Download CSVs
    snapshot_download(
        repo_id="OloriBern/FLDE",
        allow_patterns=[f"{data_name}/*.csv"],
        local_dir=path,
        revision="main",
        repo_type="dataset",
    )

    # Return csv paths (recursively)
    csv_paths = [
        os.path.join(path, data_name, file)
        for file in os.listdir(os.path.join(path, data_name))
        if file.endswith(".csv")
    ]
    return csv_paths


def download_data(pwd: str = None):
    csv_path = ft_download_data("sentence_simplification", pwd)
    data = pd.read_csv(
        csv_path[0],
        sep=" -> ",
        names=["Original sentence", "Simplified sentence"],
        header=None,
    )

    return data


# import

In [4]:
df = download_data()
df.head()

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

/tmp/ipykernel_10404/441393833.py:51: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(


,Original sentence,Simplified sentence
0,L'apprentissage des langues étrangères stimule...,On apprend mieux avec les langues étrangères.
1,Les écosystèmes marins sont régulièrement pert...,Les usines abîment souvent la vie sous la mer.
2,L'absorption de polluants atmosphériques par l...,Les forêts aident à garder l'air propre en abs...
3,"Confrontées à une mutation économique rapide, ...",Les entreprises doivent changer vite pour rest...
4,La philosophie existentialiste s'affirme par l...,L'existentialisme dit que la vie n'a pas de se...


### Download the tokenizer

In [5]:
from transformers import AutoTokenizer


def download_tokenizer(model_name: str = MODEL, training: bool = True):
    # Download tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        padding_side="left",
        truncation_side="left",
        add_eos_token=training,
        add_bos_token=True,
        trust_remote_code=True,
    )
    tokenizer.pad_token = tokenizer.eos_token

    return tokenizer


# import

In [6]:
tokenizer = download_tokenizer()

### Create the dataset

In [7]:
# ------------------------------ CREATE DATASET ------------------------------ #
from datasets import Dataset


def format_data(
    df: pd.DataFrame,
    tokenizer: AutoTokenizer,
    training: bool = True,
):
    # Create conversation
    logging.info("Create conversation...")

    def create_conversation(row):
        conversation = [
            {
                "role": "system",
                "content": "Vous êtes un modèle de langage naturel capable de simplifier des phrases en français. La phrase simplifiée doit avoir un sens aussi proche que possible de la phrase originale, mais elle est d'un niveau inférieur du CECRL et donc plus facile à comprendre. Par exemple, si une phrase est au niveau C1 du CECRL, simplifiez-la en B2. Si elle se situe au niveau B2, simplifiez-la en B1. Si elle se situe au niveau B1, simplifiez-la en A2. Si le niveau A2 est atteint, simplifiez en A1.",
            }
        ]
        if training:
            conversation.extend(
                [
                    {
                        "role": "user",
                        "content": f"""Voici une phrase en français de niveau CECRL {['A2', 'B1', 'B2', 'C1', 'C2'][row['index'] % 5]} à simplifier :
                    \"\"\"{row['Original sentence']}\"\"\"
                    Donne moi une phrase simplifiée au niveau CECRL {['A1', 'A2', 'B1', 'B2', 'C1'][row['index'] % 5]} tout en conservant au maximum son sens original
                    """,
                    },
                    {
                        "role": "assistant",
                        "content": f"{row['Simplified sentence']}",
                    },
                ]
            )
        else:
            reduced_difficulty = {
                "A1": "A1",
                "A2": "A1",
                "B1": "A2",
                "B2": "B1",
                "C1": "B2",
                "C2": "C1",
                "level1": "level1",
                "level2": "level1",
                "level3": "level2",
                "level4": "level3",
            }
            conversation.append(
                {
                    "role": "user",
                    "content": f"""Voici une phrase en français de niveau {row['Difficulty']} à simplifier :
                    \"\"\"{row['Sentence']}\"\"\"
                    Donne moi une phrase simplifiée au niveau {reduced_difficulty[row['Difficulty']]} tout en conservant au maximum son sens original
                    """,
                }
            )

        return conversation

    # Create dataset
    logging.info("Create dataset...")
    conversation_list = (
        df.reset_index()
        .apply(create_conversation, axis=1)
        .rename("conversation")
        .to_list()
    )
    dataset = Dataset.from_dict({"chat": conversation_list})

    # Format dataset
    logging.info("Format dataset...")
    formatted_dataset = dataset.map(
        lambda x: {
            "formatted_chat": tokenizer.apply_chat_template(
                x["chat"], tokenize=False, add_generation_prompt=True
            )
        }
    )

    return formatted_dataset


# import

In [8]:
dataset = format_data(df, tokenizer, training=True)
display(dataset["formatted_chat"][0])

INFO:root:Create conversation...
INFO:root:Create dataset...


INFO:root:Format dataset...


Map:   0%|          | 0/125 [00:00<?, ? examples/s]

'<s>[INST] <<SYS>>\nVous êtes un modèle de langage naturel capable de simplifier des phrases en français. La phrase simplifiée doit avoir un sens aussi proche que possible de la phrase originale, mais elle est d\'un niveau inférieur du CECRL et donc plus facile à comprendre. Par exemple, si une phrase est au niveau C1 du CECRL, simplifiez-la en B2. Si elle se situe au niveau B2, simplifiez-la en B1. Si elle se situe au niveau B1, simplifiez-la en A2. Si le niveau A2 est atteint, simplifiez en A1.\n<</SYS>>\n\nVoici une phrase en français de niveau CECRL A2 à simplifier :\n                    """L\'apprentissage des langues étrangères stimule les connexions neuronales."""\n                    Donne moi une phrase simplifiée au niveau CECRL A1 tout en conservant au maximum son sens original [/INST] On apprend mieux avec les langues étrangères. </s>'

### Tokenize the dataset

In [9]:
# ------------------------------ ENCODE DATASET ------------------------------ #
import torch
from tqdm import notebook as notebook_tqdm


def encode_dataset(dataset: Dataset, tokenizer: AutoTokenizer):
    # Determine max length
    logging.info("Determine max length...")
    max_length = max(
        [
            len(tokenizer.encode(chat))
            for chat in notebook_tqdm.tqdm(dataset["formatted_chat"])
        ]
    )

    # Encode dataset
    logging.info("Encode dataset...")
    encoded_dataset = dataset.map(
        lambda x: tokenizer(
            x["formatted_chat"],
            padding="max_length",
            truncation=True,
            max_length=max_length,
            return_attention_mask=True,
        ),
        batched=True,
    )

    # Create labels
    logging.info("Create labels...")
    encoded_dataset = encoded_dataset.map(
        lambda x: {
            "labels": x["input_ids"],
            "input_ids": x["input_ids"],
            "attention_mask": x["attention_mask"],
        },
        batched=True,
    )

    # Create dataset ready for training
    logging.info("Create dataset ready for training...")
    encoded_dataset = Dataset.from_dict(
        {
            "input_ids": torch.tensor(encoded_dataset["input_ids"]),
            "attention_mask": torch.tensor(encoded_dataset["attention_mask"]),
            "labels": torch.tensor(encoded_dataset["labels"]),
        }
    )

    # Set format
    encoded_dataset.set_format(
        type="torch",
        columns=["input_ids", "attention_mask", "labels"],
    )

    return encoded_dataset


# import

In [10]:
encoded_dataset = encode_dataset(dataset, tokenizer)
encoded_dataset

INFO:root:Determine max length...


  0%|          | 0/125 [00:00<?, ?it/s]

INFO:root:Encode dataset...


Map:   0%|          | 0/125 [00:00<?, ? examples/s]

INFO:root:Create labels...


Map:   0%|          | 0/125 [00:00<?, ? examples/s]

INFO:root:Create dataset ready for training...


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 125
})

In [11]:
# Decode dataset
display(
    pd.Series(
        encoded_dataset.map(
            lambda e: {"deencoded": tokenizer.decode(e["input_ids"])},
            remove_columns=["input_ids", "attention_mask", "labels"],
        )["deencoded"]
    )
    .apply(lambda x: x[:20])
    .value_counts()
)

pd.Series(
    encoded_dataset.map(
        lambda e: {"size": len(e["input_ids"])},
        remove_columns=["input_ids", "attention_mask", "labels"],
    )["size"]
).astype(int).describe()

Map:   0%|          | 0/125 [00:00<?, ? examples/s]

</s></s></s></s></s>    124
<s><s> [INST] <<SYS>      1
Name: count, dtype: int64

Map:   0%|          | 0/125 [00:00<?, ? examples/s]

count    125.0
mean     334.0
std        0.0
min      334.0
25%      334.0
50%      334.0
75%      334.0
max      334.0
dtype: float64